# Deep Embedding for Clustering

In this tutorial we will implement deep embedding for clustering, an end-to-end neural network solution for clustering using an autoencoder approach for network pretrainin. While the MNIST dataset is used, this implementation uses a multilayer perceptron architecture and may be generalized to any non-imaging neural network task.

Paper: https://arxiv.org/pdf/1511.06335.pdf

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from matplotlib import pyplot
from tensorflow.keras import Input, Model, layers, losses, optimizers, datasets
import tensorflow.keras.backend as K

### Data

The following code block will prepare the MNIST dataset as archived by the Tensorflow / Keras library. For purposes of demonstration, both the train and valid cohorts are combined into a single array. Additionally, the original data to type `uint8` on range `[0, 255]` is scaled to range of `[0, 1]`.

In [ ]:
def load_mnist():

    (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    
    # --- Combine train and valid for demo purposes
    x = np.concatenate((x_train, x_test))
    y = np.concatenate((y_train, y_test))
    
    # --- Flatten and scale
    x = x.reshape((x.shape[0], -1))
    x = x / 255.
    
    return x, y

In [ ]:
# --- Load data
X, y = load_mnist()

### Low-dimensional latent space visualization

At various points during this tutorial, we will examine the quality of the (low-dimensional) latent space embedding in separating out our ground-truth MNIST dataset. To do so, we will use the following code block which performs a 2-dimensional PCA and plots the results with each digit encoded in a different color.

Note that **any** valid representation of `X` may be provided into this function, including the original 784-element raw data (acknowledging that the quality of this embedding is of course limited).

In [ ]:
def pca(X, y):
    """
    Method to show PCA reduced features X
    
    :params
    
      (np.ndarray) X : 2D feature representation of size (rows, columns)
      (np.ndarray) y : 1D vector with ground-truth MNIST labels of size (rows,)
    
    """
    pyplot.clf()

    # --- Create PCA
    p = PCA(n_components=2)
    c = p.fit_transform(X)

    # --- Scatter
    fig = pyplot.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)

    for i in range(10):
        ax.scatter(c[y == i, 0], c[y == i, 1], s=2, marker='.', alpha=0.5)

    ax.axis('off')
    pyplot.show()

### K-means

In addition to viewing the quality of low-dimensional latent space embeddings, we will quantitatively evaluate the embedding in terms of separating out the ground-truth MNIST digits using a k-means clustering algorithm. The derived clusters are compared against ground-truth using an adjusted rand score (for more details see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html).  

In [ ]:
def assess_kmeans(X, y):
    
    y_ = KMeans(n_clusters=10).fit_predict(X)
    print('Adjusted rand score: {:0.4f}'.format(adjusted_rand_score(y_, y)))

# Architecture

Let us start by building a simple 2 hidden layer MLP:

In [ ]:
def create_base(shape=(784,)):
    
    x = Input(shape=shape)
    
    l0 = layers.Dense(256, activation='relu')(x)
    l1 = layers.Dense(256, activation='relu')(l0)
    
    embedding = layers.Dense(10)(l1)
    
    return Model(inputs=x, outputs=embedding)

Without further modification (e.g., after initialization with random weights), the quality of the feature embedding yielded by this MLP is poor. You can confirm this by creating this model, running a forward pass through the MNIST dataset, and evaluating the output feature representation using the `pca(...)` and `assess_kmeans(...)` methods above.

In [ ]:
# --- Create baseline (untrained) model
base = create_base()
y_ = base.predict(X)

# --- Test baseline (untrained) model representation
pca(y_, y)
assess_kmeans(y_, y)

### Autoencoder

Let us improve the quality of this feature representation using an autoencoder. The following simple architecture is symmetric to the original baseline network. 

In [ ]:
def create_autoencoder(base, shape=(784,)):
    
    x = Input(shape=shape)
    
    l0 = layers.Dense(256, activation='relu')(base(x))
    l1 = layers.Dense(256, activation='relu')(l0)
    
    recon = layers.Dense(784)(l1)
    
    return Model(inputs=x, outputs=recon)

Use the following to attach this autoencoder to the original base model:

In [ ]:
# --- Create autoencoder
base = create_base()
autoencoder = create_autoencoder(base=base)

To train, we will use standard baseline hyperparameters including an Adam optimizer with mean squared error reconstruction loss.

In [ ]:
# --- Compile and train
autoencoder.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='mse')
autoencoder.fit(x=X, y=X, batch_size=100, epochs=40)

Now let us re-test the quality of our model latent space representation:

In [ ]:
# --- Run a forward pass of data through trained encoder
y_ = base.predict(X)

In [ ]:
# --- Test baseline (trained) model representation
pca(y_, y)
assess_kmeans(y_, y)

### Deep clustering

To extend the current model into a deep clustering network, we need to create one additional special layer that will be appended to the end of our base network e.g., a *clustering* layer. Assuming a total of *k* clusters, the **weights** of this clustering layer represent the centers of the *k* different centroids. Additionally, the **output** of this clustering layer represents a probability distribution of cluster assignments e.g., the likelihood of any given single input to belong in any of *k* different clusters. More specifically, this probability distribution is modeled by a student's t-distribution as shown in the code below.

Implementation of a Keras clustering layer taken from: https://github.com/XifengGuo/DEC-keras.

In [ ]:
class ClusteringLayer(layers.Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    :params
    
      n_clusters : number of clusters.
      weights    : list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
      alpha      : parameter in Student's t-distribution. Default to 1.0.
      
    """
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
            
        super(ClusteringLayer, self).__init__(**kwargs)
        
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = layers.InputSpec(ndim=2)

    def build(self, input_shape):
        
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        
        self.input_spec = layers.InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        
        self.built = True

    def call(self, inputs, **kwargs):
        """ 
        Method to calcualte student t-distribution (same as used in t-SNE algorithm)

        :params
        
          inputs : the variable containing data, shape=(n_samples, n_features)
        
        :return
        
            q    : student's t-distribution, or soft labels for each sample of shape (n_samples, n_clusters)
            
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        
        return q

    def compute_output_shape(self, input_shape):
        
        assert input_shape and len(input_shape) == 2
        
        return input_shape[0], self.n_clusters

    def get_config(self):
        
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        
        return dict(list(base_config.items()) + list(config.items()))

Given this, use the following cell block to append a clustering layer (with 10 different clusters) to the base network pretrained from the autoencoder task:

In [ ]:
# --- Build deep clustering model
x = Input(shape=(784,))
clustering_layer = ClusteringLayer(n_clusters=10, name='clustering')(base(x))
model = Model(inputs=x, outputs=clustering_layer)

As described above, the **weights** of the clustering layer represent the cluster centroid centers. While our model will learn the optimal clusters over time, we will initiate the weight values here using the results of a naive k-means clustering:

In [ ]:
# --- Run k-means on the output of the base network
kmeans = KMeans(n_clusters=10)
kmeans.fit(base.predict(X))

In [ ]:
# --- Set weights of clustering layer
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

**Training objective**: 

In each training step, our deep clustering model will attempt to update two objectives simultaneously:

1. Improve the latent space feature representation of our base network to more closely align each training example with its closest cluster center (e.g., update to the base network weights).

2. Improve the cluster centers to better separate the training data into different groups (e.g., update to the clustering layer weights).

To train this objective, the output of the clustering layer (e.g., the predicted probability distribution) is optimized against an updated target probability distribution as shown below:

In [ ]:
def target_distribution(q):

    weight = q ** 2 / q.sum(0)

    return (weight.T / weight.sum(1)).T

This target distribution is derived in two steps:

1. Multiply the output distribution by itself (`q ** 2`): this acts to strengthen the high-probability predictions and decrease the low-probability predictions (e.g., predictions above 0.5 go towards 1.0 and those below 0.5 go towards 0.0).

2. Normalize the predictions against each cluster class: this acts to keep a relatively balanced distribution across all clusters.

### Training

Now we are set to train the deep clustering model. To do so, we will use the following steps:

1. Create a target distribution `p` based on the cluster layer output `q`.
2. Training the model for 1 epoch to based on the target distribution.
3. Visualize the new latent feature representation.
4. Repeat the loop by creating a new target distribution as in step (1).

Use the following cell to run model training:

In [ ]:
# --- Train
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='kld')

for epoch in range(20):
    
    q = model.predict(X)
    p = target_distribution(q)
    
    print('Rand score: {:0.4f}'.format(adjusted_rand_score(q.argmax(axis=1), y)))
    pca(base.predict(X), y)
    
    model.fit(x=X, y=p, batch_size=100, epochs=1)

Now let us re-test the quality of our model latent space representation:

In [ ]:
# --- Run a forward pass of data through deep clustering model
y_ = model.predict(X)

In [ ]:
# --- Test baseline (trained) model representation
pca(y_, y)
assess_kmeans(y_, y)